In [ ]:
https://tanuhack.com/statsmodels-multiple-lra/#i
#参考文献

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(font='IPAexGothic')
%config InlineBackend.figure_formats = {'png', 'retina'}
%matplotlib inline

df.plot(x='datetime', y='y', figsize=(12,5), title='時系列に沿ったお弁当の販売個数推移')
#一応、これでお弁当の販売個数推移のグラフを出すことはできるが、なぜか文字が出力されないので実行しない。

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
# @title
df[['temperature','week']]

,temperature,week
0,19.8,月
1,17.0,火
2,15.5,水
3,15.2,木
4,16.1,金
...,...,...
202,24.8,水
203,25.4,木
204,27.1,金
205,26.6,月


In [ ]:
pd.get_dummies(df[['temperature','week']])

,temperature,week_月,week_木,week_水,week_火,week_金
0,19.8,1,0,0,0,0
1,17.0,0,0,0,1,0
2,15.5,0,0,1,0,0
3,15.2,0,1,0,0,0
4,16.1,0,0,0,0,1
...,...,...,...,...,...,...
202,24.8,0,0,1,0,0
203,25.4,0,1,0,0,0
204,27.1,0,0,0,0,1
205,26.6,1,0,0,0,0


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

df = pd.read_csv('/content/drive/MyDrive/train.csv')

x = pd.get_dummies(df[['temperature','week']]) # 説明変数
y = df['y'] # 目的変数

# 定数項(y切片)を必要とする線形回帰のモデル式ならば必須
X = sm.add_constant(x)

# 最小二乗法でモデル化
model = sm.OLS(y, X)
result = model.fit()

# 重回帰分析の結果を表示する
result.summary()

#t値は、絶対値が一番大きいのはtemperatureで、「気温が上がればお弁当の販売個数は減る」ことを意味する
#p値は、0.05を超えた値は説明変数として使わない方が良い
#coefは、各説明変数の重み（係数）を示す項目。
#これらの係数は、重回帰分析モデルにおいて、各説明変数が目的変数に対してどの程度影響を与えるかを示す。
#'temperature' の係数が -2.5388 であれば、気温が1単位変化すると目的変数に対して約 -2.5388 単位の変化が期待される。

#回帰式お弁当の販売個数= 113.076 + (-2.5388 × temperature) + (30.8786 × week_月) +
#(24.4677 × week_火) + (20.5865 × week_水) + (13.1428 × week_木) + (24.0004 × week_金)

#気温20度で月曜日だった場合お弁当の販売個数= 113.076 + (-2.5388 × 20) + (30.8786 × 1)
#+ (24.4677 × 0) + (20.5865 × 0) + (13.1428 × 0) + (24.0004 × 0)=93.17859999999999
#気温20度で月曜日だった場合、93個くらい売れそうだなと予測を立てることができます！（精度44% [Adj. R-squared:	0.447] ですが笑）

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     34.30
Date:                Tue, 03 Oct 2023   Prob (F-statistic):           2.91e-25
Time:                        03:13:28   Log-Likelihood:                -952.41
No. Observations:                 207   AIC:                             1917.
Df Residuals:                     201   BIC:                             1937.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const         113.0760      3.486     32.437      0.000     106.202     119.950
temperature    -2.5388      0.198    -12.800      0.000      -2.930      -2.148
week_月         30.8786      3.569      8.651      0.000      23.841      37.917
week_木         13.1428      3.394      3.872      0.000       6.450      19.836
week_水         20.5865      3.396      6.063      0.000      13.891      27.282
week_火         24.4677      3.513      6.965      0.000      17.541      31.395
week_金         24.0004      3.496      6.866      0.000      17.107      30.893
==============================================================================
Omnibus:                       23.275   Durbin-Watson:                   1.228
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               27.801
Skew:                           0.809   Prob(JB):                     9.18e-07
Kurtosis:                       3.779   Cond. No.                     2.09e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.11e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
pred = result.predict(X)
df['pred'] = pred

df.plot(y=['y','pred'], x='datetime', figsize=(12,5), title='精度44.7％のグラフ')
#グラフ作成、文字化けあり

In [ ]:
#ステップワイズ法 STEP1：カラムが『datetime』以外のすべての説明変数を入れて重回帰分析を行う

df.fillna(0, inplace=True)
# すべての欠損値を0で置換、inplace=Trueパラメータは、変更を元のDataFrameに適用するために使用。
df.loc[df['precipitation'].str.contains('--'), 'precipitation'] = '0'
# df内からprecipitation列を取得、文字列メソッドcontainsで文字列が'--'を含むか確認。第1引数の行（df.locを使う際は、条件と考える）を第2引数の列に入れる。

x = pd.get_dummies(df[['week', 'soldout', 'name', 'kcal', 'remarks', 'event', 'payday', 'weather', 'precipitation', 'temperature']]) # 説明変数
y = df['y'] # 目的変数

# 定数項(y切片)を必要とする線形回帰のモデル式ならば必須
X = sm.add_constant(x)
model = sm.OLS(y, X)
result = model.fit()
result.summary()

# 結果がたくさん出てくる

In [ ]:
#ステップワイズ法　STEP2：p値が全て0.05(場合によって0.1)以下のもの以外を説明変数から取り除く
x = pd.get_dummies(df[['week', 'kcal', 'remarks', 'event', 'temperature']]) # 説明変数
x = x.drop(['remarks_0','remarks_手作りの味','remarks_料理長のこだわりメニュー','remarks_酢豚（28食）、カレー（85食）','remarks_鶏のレモンペッパー焼（50食）、カレー（42食）','week_木','week_金'], axis=1)
y = df['y'] # 目的変数

# 定数項(y切片)を必要とする線形回帰のモデル式ならば必須
X = sm.add_constant(x)
model = sm.OLS(y, X)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.729
Model:                            OLS   Adj. R-squared:                  0.716
Method:                 Least Squares   F-statistic:                     58.83
Date:                Tue, 03 Oct 2023   Prob (F-statistic):           4.43e-51
Time:                        04:05:20   Log-Likelihood:                -881.19
No. Observations:                 207   AIC:                             1782.
Df Residuals:                     197   BIC:                             1816.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     115.5429      3.929     29.405      0.000     107.794     123.292
kcal                       -0.0926      0.009    -10.245      0.000      -0.110      -0.075
temperature                -2.4291      0.152    -15.955      0.000      -2.729      -2.129
week_月                     19.7504      3.532      5.591      0.000      12.784      26.717
week_水                     10.1386      3.399      2.982      0.003       3.435      16.843
week_火                     13.5314      3.473      3.897      0.000       6.683      20.380
remarks_お楽しみメニュー           19.8560      6.610      3.004      0.003       6.820      32.892
remarks_スペシャルメニュー（800円）   -47.0207     20.062     -2.344      0.020     -86.585      -7.456
event_0                    38.5254      2.994     12.868      0.000      32.621      44.430
event_キャリアアップ支援セミナー        42.3059      6.915      6.118      0.000      28.669      55.943
event_ママの会                 34.7117      5.007      6.932      0.000      24.837      44.586
==============================================================================
Omnibus:                        2.957   Durbin-Watson:                   1.319
Prob(Omnibus):                  0.228   Jarque-Bera (JB):                2.964
Skew:                          -0.112   Prob(JB):                        0.227
Kurtosis:                       3.542   Cond. No.                     1.61e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.06e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# メニュー名に『カレー』を含む場合の料理の売り上げ平均
df[df['name'].str.contains('カレー')]['y'].mean()

111.17647058823529

In [ ]:
# メニュー名に『カレー』を含まない場合の料理の売り上げ平均
df[df['name'].str.match('^(?!.*カレー).*$')]['y'].mean()

84.42631578947369

In [ ]:
def curry(x):
    if 'カレー' in x:
        return 1
    else :
        return 0
# def curry(x)でカスタム関数curryを定義、引数xを受け取る。

df['curry'] = df['name'].apply(lambda x : curry(x))
# df['name].apply()で各セル値をcurry関数に渡し、その結果を新しい列curryに収納する。

# lambda キーワードは無名関数（ラムダ関数）の開始。
# x は無名関数の引数です。この引数は、関数が実行されるときに提供される値。
# :（コロン）は関数の定義と実行本体の区切りを示す。
# curry(x) は x を引数として curry 関数に渡す部分。

In [ ]:
df.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,pred,curry,year,month,fun
0,2013-11-18,90,月,0,厚切りイカフライ,0.0,0,0,0.0,快晴,0,19.8,93.686739,0,2013,11,0
1,2013-11-19,101,火,1,手作りヒレカツ,0.0,0,0,0.0,快晴,0,17.0,94.384446,0,2013,11,0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,0.0,0,0,0.0,快晴,0,15.5,94.311359,0,2013,11,0
3,2013-11-21,120,木,1,若鶏ピリ辛焼,0.0,0,0,0.0,快晴,0,15.2,87.629329,0,2013,11,0
4,2013-11-22,130,金,1,ビッグメンチカツ,0.0,0,0,0.0,快晴,0,16.1,96.202049,0,2013,11,0


In [ ]:
# remarks(特記事項)が『お楽しみメニュー』のときも売れてそう！
def fun(x):
    if x=='お楽しみメニュー':
        return 1
    else :
        return 0

df['fun'] = df['remarks'].apply(lambda x : fun(x))

In [ ]:
df.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,pred,curry,year,month,fun
0,2013-11-18,90,月,0,厚切りイカフライ,0.0,0,0,0.0,快晴,0,19.8,93.686739,0,2013,11,0
1,2013-11-19,101,火,1,手作りヒレカツ,0.0,0,0,0.0,快晴,0,17.0,94.384446,0,2013,11,0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,0.0,0,0,0.0,快晴,0,15.5,94.311359,0,2013,11,0
3,2013-11-21,120,木,1,若鶏ピリ辛焼,0.0,0,0,0.0,快晴,0,15.2,87.629329,0,2013,11,0
4,2013-11-22,130,金,1,ビッグメンチカツ,0.0,0,0,0.0,快晴,0,16.1,96.202049,0,2013,11,0


In [ ]:
# datetimeのカラムから年のカラムを作成
df['year'] = df['datetime'].apply(lambda x : x.split('-')[0])
df['year'] = df['year'].astype(int)

# datetimeのカラムから月のカラムを作成
df['month'] = df['datetime'].apply(lambda x : x.split('-')[1])
df['month'] = df['month'].astype(int)

In [ ]:
df.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,pred,curry,year,month,fun
0,2013-11-18,90,月,0,厚切りイカフライ,0.0,0,0,0.0,快晴,0,19.8,93.686739,0,2013,11,0
1,2013-11-19,101,火,1,手作りヒレカツ,0.0,0,0,0.0,快晴,0,17.0,94.384446,0,2013,11,0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,0.0,0,0,0.0,快晴,0,15.5,94.311359,0,2013,11,0
3,2013-11-21,120,木,1,若鶏ピリ辛焼,0.0,0,0,0.0,快晴,0,15.2,87.629329,0,2013,11,0
4,2013-11-22,130,金,1,ビッグメンチカツ,0.0,0,0,0.0,快晴,0,16.1,96.202049,0,2013,11,0


In [ ]:
x = pd.get_dummies(df[['year', 'month', 'week', 'kcal', 'fun', 'curry', 'weather', 'temperature']]) # 説明変数
y = df['y'] # 目的変数

# 定数項(y切片)を必要とする線形回帰のモデル式ならば必須
X = sm.add_constant(x)

# 最小二乗法でモデル化
model = sm.OLS(y, X)
result = model.fit()

# 重回帰分析の結果を表示する
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     46.49
Date:                Tue, 03 Oct 2023   Prob (F-statistic):           7.69e-57
Time:                        04:22:54   Log-Likelihood:                -851.45
No. Observations:                 207   AIC:                             1737.
Df Residuals:                     190   BIC:                             1794.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.394e+05   2.05e+04      6.812      0.000     9.9e+04     1.8e+05
year          -92.9040     13.649     -6.807      0.000    -119.827     -65.980
month          -3.5196      0.941     -3.740      0.000      -5.376      -1.663
kcal            0.0490      0.029      1.716      0.088      -0.007       0.105
fun            65.9387     12.808      5.148      0.000      40.674      91.203
curry          17.8654      4.825      3.703      0.000       8.349      27.382
temperature    -1.5291      0.279     -5.490      0.000      -2.078      -0.980
week_月       2.789e+04   4092.809      6.815      0.000    1.98e+04     3.6e+04
week_木       2.788e+04   4092.741      6.811      0.000    1.98e+04     3.6e+04
week_水       2.788e+04   4092.692      6.813      0.000    1.98e+04     3.6e+04
week_火       2.789e+04   4092.649      6.814      0.000    1.98e+04     3.6e+04
week_金       2.787e+04   4093.155      6.809      0.000    1.98e+04    3.59e+04
weather_快晴   1.992e+04   2923.095      6.814      0.000    1.42e+04    2.57e+04
weather_晴れ   1.992e+04   2923.504      6.815      0.000    1.42e+04    2.57e+04
weather_曇    1.992e+04   2923.727      6.813      0.000    1.42e+04    2.57e+04
weather_薄曇   1.992e+04   2923.139      6.816      0.000    1.42e+04    2.57e+04
weather_雨    1.992e+04   2923.802      6.814      0.000    1.42e+04    2.57e+04
weather_雪     1.99e+04   2923.663      6.807      0.000    1.41e+04    2.57e+04
weather_雷電    1.99e+04   2923.193      6.808      0.000    1.41e+04    2.57e+04
==============================================================================
Omnibus:                       12.067   Durbin-Watson:                   1.413
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               17.113
Skew:                          -0.379   Prob(JB):                     0.000192
Kurtosis:                       4.188   Cond. No.                     1.64e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.22e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# 回帰式を求める分析用のデータ
train = df.sample(frac=0.5, random_state=0)

# 回帰式の当てはまりの良さを確認するためのテストデータ
test = df.drop(train.index)

In [ ]:
# 時系列データの場合

# 回帰式を求める分析用のデータ
train = df[0:len(df)//2]

# 回帰式の当てはまりの良さを確認するためのテストデータ
test = df.drop(train.index)

In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(font='IPAexGothic')
%config InlineBackend.figure_formats = {'png', 'retina'}
%matplotlib inline

df = pd.read_csv('/content/drive/MyDrive/train.csv')

# すべての欠損値を0で置換する
df.fillna(0, inplace=True)
df.loc[df['precipitation'].str.contains('--'), 'precipitation'] = '0'

def curry(x):
    if 'カレー' in x:
        return 1
    else :
        return 0

def fun(x):
    if x=='お楽しみメニュー':
        return 1
    else :
        return 0

df['fun'] = df['remarks'].apply(lambda x : fun(x))
df['curry'] = df['name'].apply(lambda x : curry(x))

# datetimeのカラムから年のカラムを作成
df['year'] = df['datetime'].apply(lambda x : x.split('-')[0])
df['year'] = df['year'].astype(int)

# datetimeのカラムから月のカラムを作成
df['month'] = df['datetime'].apply(lambda x : x.split('-')[1])
df['month'] = df['month'].astype(int)

x = pd.get_dummies(df[['year', 'month', 'week', 'kcal', 'fun', 'curry', 'weather', 'temperature']]) # 説明変数
y = df['y'] # 目的変数

# 定数項(y切片)を必要とする線形回帰のモデル式ならば必須
X = sm.add_constant(x)

# 最小二乗法でモデル化
model = sm.OLS(y, X)
result = model.fit()

# 重回帰分析の結果を表示する
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.779
Method:                 Least Squares   F-statistic:                     46.49
Date:                Tue, 03 Oct 2023   Prob (F-statistic):           7.69e-57
Time:                        04:27:42   Log-Likelihood:                -851.45
No. Observations:                 207   AIC:                             1737.
Df Residuals:                     190   BIC:                             1794.
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        1.394e+05   2.05e+04      6.812      0.000     9.9e+04     1.8e+05
year          -92.9040     13.649     -6.807      0.000    -119.827     -65.980
month          -3.5196      0.941     -3.740      0.000      -5.376      -1.663
kcal            0.0490      0.029      1.716      0.088      -0.007       0.105
fun            65.9387     12.808      5.148      0.000      40.674      91.203
curry          17.8654      4.825      3.703      0.000       8.349      27.382
temperature    -1.5291      0.279     -5.490      0.000      -2.078      -0.980
week_月       2.789e+04   4092.809      6.815      0.000    1.98e+04     3.6e+04
week_木       2.788e+04   4092.741      6.811      0.000    1.98e+04     3.6e+04
week_水       2.788e+04   4092.692      6.813      0.000    1.98e+04     3.6e+04
week_火       2.789e+04   4092.649      6.814      0.000    1.98e+04     3.6e+04
week_金       2.787e+04   4093.155      6.809      0.000    1.98e+04    3.59e+04
weather_快晴   1.992e+04   2923.095      6.814      0.000    1.42e+04    2.57e+04
weather_晴れ   1.992e+04   2923.504      6.815      0.000    1.42e+04    2.57e+04
weather_曇    1.992e+04   2923.727      6.813      0.000    1.42e+04    2.57e+04
weather_薄曇   1.992e+04   2923.139      6.816      0.000    1.42e+04    2.57e+04
weather_雨    1.992e+04   2923.802      6.814      0.000    1.42e+04    2.57e+04
weather_雪     1.99e+04   2923.663      6.807      0.000    1.41e+04    2.57e+04
weather_雷電    1.99e+04   2923.193      6.808      0.000    1.41e+04    2.57e+04
==============================================================================
Omnibus:                       12.067   Durbin-Watson:                   1.413
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               17.113
Skew:                          -0.379   Prob(JB):                     0.000192
Kurtosis:                       4.188   Cond. No.                     1.64e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.22e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [ ]:
# テストデータと回帰式を作るデータは、全く同じ説明変数を揃えなければいけません。
#今回で言うと、curry、year、month、funを新たに説明変数として追加したので、テストデータにもそれらを追加します。
# すべての欠損値を0で置換する
test.fillna(0, inplace=True)
test.loc[test['precipitation'].str.contains('--'), 'precipitation'] = '0'

# 回帰式を作るデータと同じ説明変数を追加する
test['curry'] = test['name'].apply(lambda x : curry(x))
test['fun'] = test['remarks'].apply(lambda x : fun(x))
test['year'] = test['datetime'].apply(lambda x : x.split('-')[0])
test['year'] = test['year'].astype(int)
test['month'] = test['datetime'].apply(lambda x : x.split('-')[1])
test['month'] = test['month'].astype(int)

testX = pd.get_dummies(test[['year', 'month', 'week', 'kcal', 'fun', 'curry', 'weather', 'temperature']]) # 説明変数
testX['weather_雪'] = 0
testX['weather_雷電'] = 0
testX['const'] = 1.0

testX = testX.loc[:, X.columns]

# 重回帰式で得られた予測値をテストデータに代入させる
pred = result.predict(testX)
test['pred'] = pred


In [ ]:
test.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature,pred,curry,year,month,fun
103,2014-4-25,80,金,0,中華丼,0.0,お楽しみメニュー,0,0.0,晴れ,0,20.8,114.720850,0,2014,4,1
104,2014-4-28,54,月,0,タンドリーチキン,342.0,0,0,0.0,晴れ,0,22.7,85.638425,0,2014,4,0
105,2014-4-30,97,水,1,豚味噌メンチカツ,368.0,0,0,0.0,雨,0.5,17.5,83.156403,0,2014,4,0
106,2014-5-1,64,木,0,マーボ茄子,415.0,0,0,0.0,晴れ,0,24.0,67.178237,0,2014,5,0
107,2014-5-2,47,金,0,鶏の天ぷら,420.0,0,0,0.0,晴れ,0,23.3,62.001456,0,2014,5,0


In [ ]:
# 多重共線性を避けるために、その説明変数を除外するかしないか考えたときに参考になる指標として、
# 分散拡大係数(VIF：Variance Inflation Factor)というものがあります。
# Pythonでは、StatsModelsのメソッドstatsmodels.stats.outliers_influence.variance_inflation_factorを使って、VIFを計算できます。

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

num_cols = model.exog.shape[1] # 説明変数の列数
vifs = [vif(model.exog, i) for i in range(0, num_cols)]

pd.DataFrame(vifs, index=model.exog_names, columns=['VIF'])

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1781: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF
const,0.000000
year,18.913264
month,8.202974
kcal,18.854958
fun,7.775409
curry,1.522873
temperature,4.968543
week_月,inf
week_木,inf
week_水,inf


In [ ]:
# 一般的にVIFの値が10(公式のリファレンスでは、5)を超えると、依存関係が強いため、適切な重回帰分析ができないと言われています。
# 今回でいうと、ダミー変数化で作成した『week』の列のVIF値がすべて『inf』となっており、依存関係が非常に強いです。
# 繰り返しになりますが、重回帰分析の目的が『因果関係の洞察』であれば説明変数から除外したほうが無難であり、
# 『予測』が目的であれば除外しなくても大丈夫です。